In [3]:
import pandas as pd
import numpy as np

In [4]:
#links to download raw data from website

basics_url='https://datasets.imdbws.com/title.basics.tsv.gz'
ratings_url='https://datasets.imdbws.com/title.ratings.tsv.gz'
aka_url='https://datasets.imdbws.com/title.akas.tsv.gz'

In [27]:
#load data from local folder

basics_local='Data/title_basics.csv.gz'
ratings_local='Data/title_ratings.csv.gz'
aka_local='Data/title_akas.csv.gz'

In [7]:
basics = pd.read_csv(basics_url,sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_url,sep='\t', low_memory=False)
aka = pd.read_csv(aka_url,sep='\t', low_memory=False)

In [24]:
basics = pd.read_csv(basics_local,sep='\t', low_memory=False)
ratings = pd.read_csv(ratings_local,sep='\t', low_memory=False)
aka = pd.read_csv(aka_local,sep='\t', low_memory=False)

ParserError: Error tokenizing data. C error: Expected 1 fields in line 1097725, saw 2


In [8]:
basics.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [9]:
ratings.head()

,tconst,averageRating,numVotes
0,tt0000001,5.7,1905
1,tt0000002,5.8,256
2,tt0000003,6.5,1703
3,tt0000004,5.6,168
4,tt0000005,6.2,2518


In [10]:
aka.head()

,titleId,ordering,title,region,language,types,attributes,isOriginalTitle
0,tt0000001,1,Карменсіта,UA,\N,imdbDisplay,\N,0
1,tt0000001,2,Carmencita,DE,\N,\N,literal title,0
2,tt0000001,3,Carmencita - spanyol tánc,HU,\N,imdbDisplay,\N,0
3,tt0000001,4,Καρμενσίτα,GR,\N,imdbDisplay,\N,0
4,tt0000001,5,Карменсита,RU,\N,imdbDisplay,\N,0


In [11]:
basics.shape

(9163395, 9)

In [12]:
ratings.shape

(1255994, 3)

In [13]:
aka.shape

(32914228, 8)

In [14]:
#replacing charactes \N with python Nulls
basics.replace({'\\N':np.nan},inplace=True)
ratings.replace({'\\N':np.nan},inplace=True)
aka.replace({'\\N':np.nan},inplace=True)

In [15]:
#filter for aka dataframe

aka_filter_region=aka['region']=="US"

#Creating aka filtered DF

df_aka=aka[aka_filter_region]


In [20]:
basics['startYear'] = basics['startYear'].astype(float)

In [21]:
#setting filters for Basic DF based on project requirements

basics_filter_nulls= pd.notnull(basics["runtimeMinutes"]) & pd.notnull(basics["genres"])
basics_filter_movie= basics['titleType']=='movie'
basics_filter_syear= (basics['startYear']>=2000) & (basics['startYear']<=2022)
basics_filter_documentary= ~(basics['genres'].str.contains('documentary',case=False,na=False))
basics_filter_region=basics['tconst'].isin(df_aka['titleId'])

#creating basics filtered DF
df_basics=basics[basics_filter_nulls&basics_filter_movie&basics_filter_syear&basics_filter_documentary&basics_filter_region]


In [22]:
df_basics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 82213 entries, 34790 to 9163160
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   tconst          82213 non-null  object 
 1   titleType       82213 non-null  object 
 2   primaryTitle    82213 non-null  object 
 3   originalTitle   82213 non-null  object 
 4   isAdult         82213 non-null  object 
 5   startYear       82213 non-null  float64
 6   endYear         0 non-null      object 
 7   runtimeMinutes  82213 non-null  object 
 8   genres          82213 non-null  object 
dtypes: float64(1), object(8)
memory usage: 6.3+ MB


In [23]:
df_aka.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1343796 entries, 5 to 32913972
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1343796 non-null  object
 1   ordering         1343796 non-null  int64 
 2   title            1343796 non-null  object
 3   region           1343796 non-null  object
 4   language         3685 non-null     object
 5   types            963507 non-null   object
 6   attributes       44791 non-null    object
 7   isOriginalTitle  1342421 non-null  object
dtypes: int64(1), object(7)
memory usage: 92.3+ MB


In [24]:
#Ratings didn't need any addtional data preparation
df_ratings=ratings
df_ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1255994 entries, 0 to 1255993
Data columns (total 3 columns):
 #   Column         Non-Null Count    Dtype  
---  ------         --------------    -----  
 0   tconst         1255994 non-null  object 
 1   averageRating  1255994 non-null  float64
 2   numVotes       1255994 non-null  int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 28.7+ MB


In [25]:
# creating folder to save files from website
import os
os.makedirs('Data/',exist_ok=True) 
# Confirm folder created
os.listdir("Data/")


['title.akas.tsv.gz',
 'title.ratings.tsv.gz',
 'title_basics.csv',
 'title_basics.csv.gz']

In [96]:
#savings original files from website

basics.to_csv("Data/title_basics.csv.gz",compression='gzip',index=False)
aka.to_csv("Data/title_akas.csv.gz",compression='gzip',index=False)
ratings.to_csv("Data/title_ratings.csv.gz",compression='gzip',index=False)

In [97]:
df_basics.to_csv("Data/title_basics_filtered.csv.gz",compression='gzip',index=False)
df_aka.to_csv("Data/title_akas_filtered.csv.gz",compression='gzip',index=False)
df_ratings.to_csv("Data/title_ratings_filtered.csv.gz",compression='gzip',index=False)

In [31]:
df_aka.groupby('region').count()

,titleId,ordering,title,language,types,attributes,isOriginalTitle
region,,,,,,,
US,1343796,1343796,1343796,3685,963507,44791,1342421


In [32]:
df_aka.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1343796 entries, 5 to 32913972
Data columns (total 8 columns):
 #   Column           Non-Null Count    Dtype 
---  ------           --------------    ----- 
 0   titleId          1343796 non-null  object
 1   ordering         1343796 non-null  int64 
 2   title            1343796 non-null  object
 3   region           1343796 non-null  object
 4   language         3685 non-null     object
 5   types            963507 non-null   object
 6   attributes       44791 non-null    object
 7   isOriginalTitle  1342421 non-null  object
dtypes: int64(1), object(7)
memory usage: 92.3+ MB


In [88]:
import os, time,json
import tmdbsimple as tmdb 
from tqdm.notebook import tqdm_notebook
FOLDER = "Data/"

In [90]:
with open('/Users/JoseH/.secret/tmdb_api.json', 'r') as f:
    login = json.load(f)
## Display the keys of the loaded dict
login.keys()


dict_keys(['client-id', 'api-key'])

In [91]:
tmdb.API_KEY =  login['api-key']

In [92]:
def write_json(new_data, filename): 
    """Adapted from: https://www.geeksforgeeks.org/append-to-json-file-using-python/"""    
    
    with open(filename,'r+') as file:
        # First we load existing data into a dict.
        file_data = json.load(file)
        ## Choose extend or append
        if (type(new_data) == list) & (type(file_data) == list):
            file_data.extend(new_data)
        else:
             file_data.append(new_data)
        # Sets file's current position at offset.
        file.seek(0)
        # convert back to json.
        json.dump(file_data, file)

In [93]:
def movie_data(id):
    # Get the movie object for the current id
    movie = tmdb.Movies(id)
    # save the .info .releases dictionaries
    info = movie.info()
    releases = movie.releases()
    # Loop through countries in releases
    for c in releases['countries']:
        # if the country abbreviation==US
        if c['iso_3166_1' ] =='US':
            ## save a "certification" key in the info dict with the certification
           info['certification'] = c['certification']
    return info

In [102]:
avengers=movie_data('tt0848228')
notebook=movie_data('tt0332280')

In [103]:
avengers

{'adult': False,
 'backdrop_path': '/nNmJRkg8wWnRmzQDe2FwKbPIsJV.jpg',
 'belongs_to_collection': {'id': 86311,
  'name': 'The Avengers Collection',
  'poster_path': '/yFSIUVTCvgYrpalUktulvk3Gi5Y.jpg',
  'backdrop_path': '/zuW6fOiusv4X9nnW3paHGfXcSll.jpg'},
 'budget': 220000000,
 'genres': [{'id': 878, 'name': 'Science Fiction'},
  {'id': 28, 'name': 'Action'},
  {'id': 12, 'name': 'Adventure'}],
 'homepage': 'https://www.marvel.com/movies/the-avengers',
 'id': 24428,
 'imdb_id': 'tt0848228',
 'original_language': 'en',
 'original_title': 'The Avengers',
 'overview': 'When an unexpected enemy emerges and threatens global safety and security, Nick Fury, director of the international peacekeeping agency known as S.H.I.E.L.D., finds himself in need of a team to pull the world back from the brink of disaster. Spanning the globe, a daring recruitment effort begins!',
 'popularity': 237.136,
 'poster_path': '/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg',
 'production_companies': [{'id': 420,
   'logo_path

In [104]:
notebook

{'adult': False,
 'backdrop_path': '/qom1SZSENdmHFNZBXbtJAU0WTlC.jpg',
 'belongs_to_collection': None,
 'budget': 29000000,
 'genres': [{'id': 10749, 'name': 'Romance'}, {'id': 18, 'name': 'Drama'}],
 'homepage': 'http://www.newline.com/properties/notebookthe.html',
 'id': 11036,
 'imdb_id': 'tt0332280',
 'original_language': 'en',
 'original_title': 'The Notebook',
 'overview': "An epic love story centered around an older man who reads aloud to a woman with Alzheimer's. From a faded notebook, the old man's words bring to life the story about a couple who is separated by World War II, and is then passionately reunited, seven years later, after they have taken different paths.",
 'popularity': 70.493,
 'poster_path': '/rNzQyW4f8B8cQeg7Dgj3n6eT5k9.jpg',
 'production_companies': [{'id': 12,
   'logo_path': '/iaYpEp3LQmb8AfAtmTvpqd4149c.png',
   'name': 'New Line Cinema',
   'origin_country': 'US'},
  {'id': 1565, 'logo_path': None, 'name': 'Avery Pix', 'origin_country': 'US'},
  {'id': 26

In [94]:
years=[2000,2001]

In [95]:
for YEAR in tqdm_notebook(years,desc='YEARS',position=0):
    JSON_FILE = f'{FOLDER}tmdb_api_results_{YEAR}.json'
    file_exists = os.path.isfile(JSON_FILE)
    # If it does not exist: create it
    if file_exists == False:
        # save an empty dict with just "imdb_id" to the new json file.
        with open(JSON_FILE,'w') as f:
            json.dump([{'imdb_id':0}],f)
    #Saving new year as the current df
    df = df_basics.loc[ basics['startYear']==YEAR].copy()
    # saving movie ids to list
    movie_ids = df['tconst'].copy()#.to_list()
    # Load existing data from json into a dataframe called "previous_df"
    previous_df = pd.read_json(JSON_FILE)
    # filter out any ids that are already in the JSON_FILE
    movie_ids_to_get = movie_ids[~movie_ids.isin(previous_df['imdb_id'])]
    
    for movie_id in tqdm_notebook(movie_ids_to_get,
                                  desc=f'Movies from {YEAR}',
                                  position=1,
                                  leave=True):
        # Attempt to retrieve then data for the movie id
        try:
            temp = movie_data(movie_id)  #This uses your pre-made function
            # Append/extend results to existing file using a pre-made function
            write_json(temp,JSON_FILE)
            # Short 20 ms sleep to prevent overwhelming server
            time.sleep(0.02)
            
        # If it fails,  make a dict with just the id and None for certification.
        except Exception as e:
            continue
    final_year_df = pd.read_json(JSON_FILE)
    final_year_df.to_csv(f"{FOLDER}final_tmdb_data_{YEAR}.csv.gz", compression="gzip", index=False)

YEARS:   0%|          | 0/2 [00:00<?, ?it/s]

Movies from 2000:   0%|          | 0/1408 [00:00<?, ?it/s]

Movies from 2001:   0%|          | 0/1529 [00:00<?, ?it/s]